## promise patterns


### resolve a promise property?

 TODO: make this even simpler, if working with promises then it can callback on it own similar to lazy loading



In [ ]:

function promiseOrResolve(obj, property, cb) {
    return obj ? (typeof obj[property] != 'undefined'
                  ? Promise.resolve(obj[property])
                  : cb(obj).then(d => (obj[property] = d)))
        : Promise.resolve()
}

module.exports = promiseOrResolve;


## classes and objects

Long term goal: use this notebook at training data to mangle/uglify/transpile any language

TODO: import falsey javascript in to this notebook?

demonstrate patterns between different languages, in C# interfaces are like:

```
interface MyInterface {
    myProperty: string
}

class MyClass : MyInterface {
}
```




### making a class the normal way?


In [ ]:
if(typeof parentClass === 'undefined') {
    class parentClass {
        constructor() {
            console.log('parent');
        }

        do_message() {
            console.log('original');
        }
    }

    class childClass extends parentClass {
        constructor() {
            console.log('child');
            super();
        }

        do_message() {
            super.do_message();
            console.log('overridden');
        }
    }
}

// TODO: better way to do this, childClass.prototype.bind?
module.exports = () => new childClass;

if(typeof $$ !== 'undefined') {
    console.log(typeof module.exports);
    var testClass = module.exports();
    testClass.do_message();
    
    /* expected output
    function
    child
    parent
    original
    overridden
    */
    
    /*
    this is extremely annoying, how to achieve the same effect with prototype?
    the javascript language is apparently not indempotent
    
    evalmachine.<anonymous>:1
    class parentClass {
    ^

    SyntaxError: Identifier 'parentClass' has already been declared
        at evalmachine.<anonymous>:1:1
    */
}




### extend prototype class?

derived from to help avoid annoying notebook errors:

https://stackoverflow.com/questions/10430279/extending-an-object-in-javascript


In [72]:
var parentObj = {
    do_init() {
        console.log('parent');
    },
    do_message() {
        console.log('original');
    }
}

var childObj = {
    do_init() {
        console.log('child');
        parentObj.do_init();
    },
    do_message() {
        parentObj.do_message();
        console.log('overridden');
    }
}

function extend(child, parent) {
    var newClass = Object.create(parent || {});
    Object.assign(newClass, child, Object.getPrototypeOf(child || {}));
    var self = Object.create(newClass);
    return self;
}

function override(child) {
    return extend(child, this);
}

module.exports = {
    extend,
    override
}
//module.exports = Object.create.bind(null, childObj)

if(typeof $$ !== 'undefined') {
    console.log(typeof module.exports)
    var testClass = module.exports.extend();
    console.log(typeof testClass.do_init)
    testClass = module.exports.extend(childObj, parentObj)
    testClass.do_init();
    testClass.do_message();
    testClass = module.exports.extend({}, parentObj)
    testClass.do_init();
    testClass.do_message();
    // TODO: more tests
    
    /* expected output
    object
    undefined
    child
    parent
    original
    overridden
    parent
    original
    */
}



object
undefined
child
parent
original
overridden
parent
original


### enforcing an interface?


In [75]:

// newer ES6 syntax

function typeErrorTemplate(e, k, t, i, p) {
    if(k) {
        k = ' ' + k;
    }
    if(p) {
        p = ' of type ' + p;
    }
    throw new Error(`type mis-match${k || ''}: "${t}" is not "${i}"${p || ''}`, e)
}

function standardCompare(type, expected) {
    if(type === expected) return true;
    if(!type || !expected || !(expected.isPrototypeOf(type))) {
        return false
    }
    return true;
}

function arrayCompare(compare, specification, loosey) {
    var match = specification.map(i => {
        try {
            return interface(compare, i, loosey);
        } catch (e) {
            return e;
        }
    }).filter(s => !s || s.constructor !== Error);
    if(match.length > 0) {
        return match[0];
    }
    typeErrorTemplate(void 0,
                      void 0,
                      typeof compare,
                      specification.map(s => typeof s));
}

function objectCompare(compare, specification, loosey) {
    var match = Object.keys(specification).reduce((map, k) => {
        try {
            var m = interface(compare[k], specification[k], loosey);
            if(k === 'kernel_config') {
            }
            if(typeof m !== 'undefined') {
                map[k] = m;
            }
        } catch (e) {
            typeErrorTemplate(e,
                              k,
                              typeof compare[k],
                              specification[k],
                              specification[k].constructor)
        }
        return map;
    }, {});
    return match;
}

// loosey means no exceptions are thrown and undefined type is assumed
function interface(compare, specification, loosey) {
    var type = compare === void 0 || compare === null
        ? compare
        : Object.getPrototypeOf(compare);
    var expected = specification === void 0 || specification === null
        ? specification
        : Object.getPrototypeOf(specification);
    switch(expected) {
        case Array.prototype:
            // if it is an empty array to the actual compare on the object,
            //   if it is not an empty array compare to each type in the array
            if(specification.length > 0) {
                return arrayCompare(compare, specification, loosey)
            }
        case Object.prototype:
            // compare properties on object
            if(Object.keys(specification).length > 0
               && (typeof compare === 'object' || typeof compare === 'function')) {
               return objectCompare(compare, specification, loosey)
            }
        default:
            if(standardCompare(type, expected)
               // if loosey is not explicitly set to false,
               //   return void 0 (undefined) instead of error
               //   only if compare it undefined,
               //   still error on other wrong types
               || (loosey !== false && compare === void 0)) {
                return compare;
            }
            typeErrorTemplate(void 0,
                              void 0,
                              typeof compare,
                              typeof specification,
                              expected)
    }
}

module.exports = interface;

if(typeof $$ !== 'undefined') {
    var jsonInterface = {
        display_name: '',
        argv: [], // not optional
        language: '', // not optional
        metadata: [void 0, {}], // optional
        env: [void 0, {}], // TODO dictionary descriptor types?
        interrupt_mode: [void 0, '']
    }

    console.log(interface({
        display_name: 'Node JS',
        argv: [],
        language: '',
        some_other_stuff: true
    }, jsonInterface))
    
    // expected output {"display_name":"Node JS","argv":[],"language":""}
}



{ display_name: 'Node JS', argv: [], language: '' }


#### test enforcing an interface with prototype?



In [76]:

function myInterface(overrides) {
    var interface = Object.create({});
    // TODO: wish there was a way to not name this twice
    Object.assign(interface, {
        propertyOne: overrides.propertyOne,
        print: overrides.print
    });
    return interface;
}

function print() {
    console.log(this.propertyOne);
    console.log(this.propertyTwo);
}

var myClass = Object.create({
    propertyOne: 'original 1',
    propertyTwo: 'original 2',
    print: print
})

if(typeof $$ !== 'undefined') {

    myClass.print();
    var overridden = overrideClass({
        propertyOne: 'overridden 1',
        propertyTwo: 'overridden 2'
    })
    overridden.print();
    var interfaced = myInterface(overridden);
    interfaced.print();
    overrideClass(interfaced).print();
    myInterface(myClass).print();
    
}

/* expected output
original 1
original 2
overridden 1
overridden 2
overridden 1
undefined
overridden 1
original 2
original 1
undefined
*/



original 1
original 2
overridden 1
overridden 2
overridden 1
undefined
overridden 1
original 2
original 1
undefined
